## Stewart: calculate velocities

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import scvelo as scv

### colormap for UMAP

In [ ]:
# list of HEX colors
hex_colors = ["#66BA7B", "#C08C1C", "#CB67EF", "#DC726C", "#75C1C3", "#7DB93F", "#DF60B5", "#9390F3", "#5FACF2", "#9EA022"]

# convert HEX colors to RGB
rgb_colors = [(int(color[1:3], 16)/255, int(color[3:5], 16)/255, int(color[5:7], 16)/255) for color in hex_colors]

# create colormap
cmap = ListedColormap(rgb_colors)

# create dictionary to assign colors to clusters
clusters = ["Trans", "Naive", "M-mem1", "M-mem2", "C-mem1", "C-mem2", "DN1", "DN2", "DN3", "DN4"]
umap_colors = {}
i = 0
for cluster in clusters:
    umap_colors[cluster] = cmap.colors[i]
    i+=1

### read in data

cellranger/velocyto files

In [ ]:
# adata file from cellranger
cellranger = sc.read_h5ad("../data/stewart_cellranger.h5ad")

In [ ]:
# loom files from velocyto
ldata_naive = scv.read("../cellranger/naive/velocyto/naive.loom", cache=True)
ldata_trans = scv.read("../cellranger/trans/velocyto/trans.loom", cache=True)
ldata_igmmem = scv.read("../cellranger/igmmem/velocyto/igmmem.loom", cache=True)
ldata_classical = scv.read("../cellranger/classical/velocyto/classical.loom", cache=True)
ldata_dn = scv.read("../cellranger/dn/velocyto/dn.loom", cache=True)

In [ ]:
# rename barcodes
ldata_classical.obs_names = ldata_classical.obs_names.str.replace("x", "-1").str.replace("classical", "").str.replace(":", "")
ldata_naive.obs_names = ldata_naive.obs_names.str.replace("x", "-2").str.replace("naive", "").str.replace(":", "")
ldata_trans.obs_names = ldata_trans.obs_names.str.replace("x", "-3").str.replace("trans", "").str.replace(":", "")
ldata_igmmem.obs_names = ldata_igmmem.obs_names.str.replace("x", "-4").str.replace("igmmem", "").str.replace(":", "")
ldata_dn.obs_names = ldata_dn.obs_names.str.replace("x", "-5").str.replace("dn", "").str.replace(":", "")

In [ ]:
# merge adata with loom file generated by velocyto
naive = scv.utils.merge(cellranger[cellranger.obs['population'] == 'Naive'], ldata_naive)
trans = scv.utils.merge(cellranger[cellranger.obs['population'] == 'Transitional'], ldata_trans)
igmmem = scv.utils.merge(cellranger[cellranger.obs['population'] == 'IgM-Memory'], ldata_igmmem)
classical = scv.utils.merge(cellranger[cellranger.obs['population'] == 'Classical-Memory'], ldata_classical)
dn = scv.utils.merge(cellranger[cellranger.obs['population'] == 'DN'], ldata_dn)

In [ ]:
# concatenate samples
velocyto = classical.concatenate(naive, trans, igmmem, dn)
velocyto.obs_names = [name[:-2] for name in velocyto.obs_names]

alevin-fry file

In [ ]:
alevin = sc.read_h5ad("../data/stewart_salmon.h5ad")

### compute velocities

In [ ]:
# preprocessing
scv.pp.filter_genes(velocyto)
scv.pp.normalize_per_cell(velocyto)

scv.pp.filter_genes(alevin)
scv.pp.normalize_per_cell(alevin)

# ensure that highly variable genes are the same
assert (velocyto.var["highly_variable"].sort_index()).equals(alevin.var["highly_variable"].sort_index()), "computing velocities on different genes"

In [ ]:
# compute neighborhood graph and moments for velocity estimation
scv.pp.moments(velocyto)
scv.pp.moments(alevin)

In [ ]:
# separate into 3 objects for 3 modes (deterministic, stochastic, dynamical)
velocyto_det = velocyto.copy()
velocyto_sto = velocyto.copy()
velocyto_dyn = velocyto.copy()

alevin_det = alevin.copy()
alevin_sto = alevin.copy()
alevin_dyn = alevin.copy()

#### model = deterministic

In [ ]:
# calculate velocities: deterministic (= steady state = velocyto)
scv.tl.velocity(velocyto_det, mode='deterministic', filter_genes=False)
scv.tl.velocity_graph(velocyto_det)

scv.tl.velocity(alevin_det, mode='deterministic', filter_genes=False)
scv.tl.velocity_graph(alevin_det)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_det, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_det, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

#### model = stochastic

In [ ]:
# calculate velocities: stochastic 
scv.tl.velocity(velocyto_sto, mode='stochastic', filter_genes=False)
scv.tl.velocity_graph(velocyto_sto)

scv.tl.velocity(alevin_sto, mode='stochastic', filter_genes=False)
scv.tl.velocity_graph(alevin_sto)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_sto, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_sto, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

#### model = dynamical

In [ ]:
# calculate velocities: dynamical 
scv.tl.recover_dynamics(velocyto_dyn)
scv.tl.velocity(velocyto_dyn, mode="dynamical", filter_genes=False)

scv.tl.recover_dynamics(alevin_dyn)
scv.tl.velocity(alevin_dyn, mode="dynamical", filter_genes=False)

In [ ]:
# velocity graph
scv.tl.velocity_graph(velocyto_dyn)

scv.tl.velocity_graph(alevin_dyn)

In [ ]:
# embedding in UMAP
scv.pl.velocity_embedding_grid(velocyto_dyn, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

scv.pl.velocity_embedding_grid(alevin_dyn, basis='umap', color="cluster", arrow_length = 2, palette=umap_colors,
                               legend_loc = 'right_margin', xlabel = "UMAP 1", ylabel = "UMAP 2")

### save

In [ ]:
velocyto_sto.write_h5ad(filename="../data/stewart_velocyto.h5ad")
alevin_sto.write_h5ad(filename="../data/stewart_alevin.h5ad")